# Import Libs

In [1]:
import pandas as pd                                           # for create Excel Sheet
import requests                                               # for send the request of url
from bs4 import BeautifulSoup as Bs                           # for web scraping
import re                                                     # for text pattern extraction 
import pygsheets                                              # for interaction to Google Sheets

# Task is extracting table from wikipedia link, we will use BeautifulSoup

In [2]:
##URL
url = 'https://ar.wikipedia.org/wiki/%D9%82%D8%A7%D8%A6%D9%85%D8%A9_%D8%A3%D9%81%D8%B6%D9%84_%D9%85%D8%A6%D8%A9_%D8%B1%D9%88%D8%A7%D9%8A%D8%A9_%D8%B9%D8%B1%D8%A8%D9%8A%D8%A9'

##send request
req = requests.get(url)

##get the content
soup = Bs(req.text,'html.parser')


# Start Scraping Table

In [3]:
#create list that contain the scraped data
index_list=[]
book_title_list=[]
book_url_list=[]
author_name_list=[]
author_url_list=[]
country_name_list=[]
country_url_list=[]

## get Table
table = soup.find('table', class_='wikitable sortable')

##get table rows
rows = table.find_all('tr')

##get rows data
for i in range(1,len(rows)):
    #total row data
    data = rows[i].find_all('td')
    
    #get the index "replace unneeded charcters'\n'" ---> Example '1\n'->'1'
    index =data[0].text.replace('\n','')
    index_list.append(index)
    
    #get title text "replace unneeded charcters'\n'" ---> Example 'الثلاثية\n'->'الثلاثية'
    book_title = data[1].text.replace('\n','')
    book_title_list.append(book_title)
    
    #get book url "get the pattern of url using reqular experssion"
    # from first galance the book_url is shorted link we need to modefied it
    # example
    #shorted '/wiki/%D8%A7%D9%84%D8%A8%D8%A7%D8%A8_%D8%A7%D9'
    #complete https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%A8%D8%A7%D8%A8_%D8%A7%D9
    book_url_att = data[1].find('a')
    book_url_att_mod = re.findall("href=[\"\'](.*?)[\"\']", str(book_url_att))[0]
    
    if(bool(re.findall('/wiki/',book_url_att_mod))):
        book_url = book_url_att_mod.replace('/wiki/','https://ar.wikipedia.org/wiki/',1)
    else:
        book_url= book_url_att_mod.replace("/w/","https://ar.wikipedia.org/w/",1)

    book_url_list.append(book_url)
    
    #get author text
    author = data[2].text.replace('\n','')
    author_name_list.append(author)
    
    #get author url "get the pattern of url using reqular experssion"
    # from first galance the author_url is shorted link we need to modefied it
    # example
    #shorted '/wiki/%D8%A7%D9%84%D8%A8%D8%A7%D8%A8_%D8%A7%D9'
    #complete https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%A8%D8%A7%D8%A8_%D8%A7%D9
    author_url_att = data[2].find('a')
    author_url_att_mod = re.findall("href=[\"\'](.*?)[\"\']", str(author_url_att))[0]
    
    if(bool(re.findall('/wiki/',author_url_att_mod))):
        author_url = author_url_att_mod.replace('/wiki/','https://ar.wikipedia.org/wiki/',1)
    else:
        author_url= author_url_att_mod.replace("/w/","https://ar.wikipedia.org/w/",1)
    
    author_url_list.append(author_url)
    
    #get country text
    country = data[3].text.replace('\n','').replace('\xa0','')
    country_name_list.append(country)
    
    #get country image url
    country_url_att = data[3].find('img')
    country_url = re.findall("src=[\"\'](.*?)[\"\']", str(country_url_att))[0]
    country_url_list.append(country_url)

In [4]:
extracted_table = pd.DataFrame(list(zip(index_list, book_title_list, book_url_list, author_name_list, author_url_list, country_name_list, country_url_list)),
               columns =['index', 'book_title', 'book_url', 'author_name', 'author_url', 'country_name', 'country_url'])

# Extracted_table

In [5]:
extracted_table

,index,book_title,book_url,author_name,author_url,country_name,country_url
0,1,الثلاثية,https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%...,نجيب محفوظ,https://ar.wikipedia.org/wiki/%D9%86%D8%AC%D9%...,مصر,//upload.wikimedia.org/wikipedia/commons/thumb...
1,2,البحث عن وليد مسعود,https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%...,جبرا إبراهيم جبرا,https://ar.wikipedia.org/wiki/%D8%AC%D8%A8%D8%...,فلسطين,//upload.wikimedia.org/wikipedia/commons/thumb...
2,3,شرف,https://ar.wikipedia.org/wiki/%D8%B4%D8%B1%D9%...,صنع الله إبراهيم,https://ar.wikipedia.org/wiki/%D8%B5%D9%86%D8%...,مصر,//upload.wikimedia.org/wikipedia/commons/thumb...
3,4,الحرب في بر مصر,https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%...,يوسف القعيد,https://ar.wikipedia.org/wiki/%D9%8A%D9%88%D8%...,مصر,//upload.wikimedia.org/wikipedia/commons/thumb...
4,5,رجال في الشمس,https://ar.wikipedia.org/wiki/%D8%B1%D8%AC%D8%...,غسان كنفاني,https://ar.wikipedia.org/wiki/%D8%BA%D8%B3%D8%...,فلسطين,//upload.wikimedia.org/wikipedia/commons/thumb...
...,...,...,...,...,...,...,...
101,102,الفارس القتيل يترجل,https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D9%...,إلياس الديري,https://ar.wikipedia.org/wiki/%D8%A5%D9%84%D9%...,لبنان,//upload.wikimedia.org/wikipedia/commons/thumb...
102,103,التوت المر,https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%...,محمد العروسي المطوي,https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%...,تونس,//upload.wikimedia.org/wikipedia/commons/thumb...
103,104,أغنية الماء والنار,https://ar.wikipedia.org/wiki/%D8%A3%D8%BA%D9%...,عبد الله خليفة,https://ar.wikipedia.org/wiki/%D8%B9%D8%A8%D8%...,البحرين,//upload.wikimedia.org/wikipedia/commons/thumb...
104,105,الباب المفتوح,https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%...,لطيفة الزيات,https://ar.wikipedia.org/wiki/%D9%84%D8%B7%D9%...,مصر,//upload.wikimedia.org/wikipedia/commons/thumb...


# Save Table to Excel

In [6]:
extracted_table.to_excel('../Output/Scraped_output.xlsx',index=False)

# Save Table to Google Sheet

In [7]:
# This will create a link to authorize 
gc = pygsheets.authorize(service_account_file='../Files/mohamed-334615-da3d205fd31f.json') 

#  Open spreadsheet  
sh = gc.open_by_key('1Q-SA6PGA6xRe3HzEEmNOnvClU3mhemSKBLSrg_g7PdU')

# Open worksheet
wk1 = sh[0]

# Link : https://docs.google.com/spreadsheets/d/1Q-SA6PGA6xRe3HzEEmNOnvClU3mhemSKBLSrg_g7PdU/edit?usp=sharing
wk1.set_dataframe(extracted_table, 'A1')
